In [21]:
import numpy as np
import pandas as pd
from google.oauth2 import service_account
from apiclient.discovery import build

In [14]:
def format_summary(response):
    try:
        # create row index
        try: 
            row_index_names = response['reports'][0]['columnHeader']['dimensions']
            row_index = [ element['dimensions'] for element in response['reports'][0]['data']['rows'] ]
            row_index_named = pd.MultiIndex.from_arrays(np.transpose(np.array(row_index)), 
                                                        names = np.array(row_index_names))
        except:
            row_index_named = None
        
        # extract column names
        summary_column_names = [item['name'] for item in response['reports'][0]
                                ['columnHeader']['metricHeader']['metricHeaderEntries']]
    
        # extract table values
        summary_values = [element['metrics'][0]['values'] for element in response['reports'][0]['data']['rows']]
    
        # combine. I used type 'float' because default is object, and as far as I know, all values are numeric
        df = pd.DataFrame(data = np.array(summary_values), 
                          index = row_index_named, 
                          columns = summary_column_names).astype('float')
    
    except:
        df = pd.DataFrame()
        
    return df

def format_pivot(response):
    try:
        # extract table values
        pivot_values = [item['metrics'][0]['pivotValueRegions'][0]['values'] for item in response['reports'][0]
                        ['data']['rows']]
        
        # create column index
        top_header = [item['dimensionValues'] for item in response['reports'][0]
                      ['columnHeader']['metricHeader']['pivotHeaders'][0]['pivotHeaderEntries']]
        column_metrics = [item['metric']['name'] for item in response['reports'][0]
                          ['columnHeader']['metricHeader']['pivotHeaders'][0]['pivotHeaderEntries']]
        array = np.concatenate((np.array(top_header),
                                np.array(column_metrics).reshape((len(column_metrics),1))), 
                               axis = 1)
        column_index = pd.MultiIndex.from_arrays(np.transpose(array))
        
        # create row index
        try:
            row_index_names = response['reports'][0]['columnHeader']['dimensions']
            row_index = [ element['dimensions'] for element in response['reports'][0]['data']['rows'] ]
            row_index_named = pd.MultiIndex.from_arrays(np.transpose(np.array(row_index)), 
                                                        names = np.array(row_index_names))
        except: 
            row_index_named = None
        # combine into a dataframe
        df = pd.DataFrame(data = np.array(pivot_values), 
                          index = row_index_named, 
                          columns = column_index).astype('float')
    except:
        df = pd.DataFrame()
    return df

def format_report(response):
    summary = format_summary(response)
    pivot = format_pivot(response)
    if pivot.columns.nlevels == 2:
        summary.columns = [['']*len(summary.columns), summary.columns]
    
    return(pd.concat([summary, pivot], axis = 1))

def run_report(body, credentials_file):
    #Create service credentials
    credentials = service_account.Credentials.from_service_account_file(credentials_file, 
                                scopes = ['https://www.googleapis.com/auth/analytics.readonly'])
    #Create a service object
    service = build('analyticsreporting', 'v4', credentials=credentials)
    
    #Get GA data
    response = service.reports().batchGet(body=body).execute()
    
    return(format_report(response))

In [18]:
your_view_id = '230256006'
ga_keys = 'ethereal-casing-320607-af4cd783985b.json'

In [22]:
body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2021-01-01', 'endDate': '2021-07-22'}],
                            'metrics': [{'expression': 'ga:users'}, 
                                        {"expression": "ga:bounceRate"}],
                            'dimensions': [{'name': 'ga:yearMonth'}],
                            "pivots": [{"dimensions": [{"name": "ga:channelGrouping"}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:bounceRate"}]
                                       }]
                          }]}

In [ ]:
summary_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2021-01-01', 'endDate': '2021-02-28'}],
                            'metrics': [{'expression': 'ga:sessions'}, 
                                        {'expression': 'ga:totalEvents'}, 
                                        {"expression": "ga:avgSessionDuration"}],
                            'dimensions': [{'name': 'ga:country'}],
                          }]}

In [ ]:
pivot_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2021-01-01', 'endDate': '2021-02-28'}],
                            'dimensions': [{'name':  "ga:channelGrouping"}],
                            "pivots": [{"dimensions": [{"name": 'ga:yearMonth'}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:newUsers"},
                                                    {"expression": "ga:timeOnPage"}]
                                       }]
                          }]}

In [ ]:
short_body = {  "reportRequests":
  [{
      "viewId": your_view_id,
      "dateRanges": [{"startDate": "7daysAgo", "endDate": "yesterday"}],
      "metrics": [{"expression": "ga:users"}]
    }]}

In [24]:
untidy_body = {'reportRequests': [{'viewId': your_view_id, 
                            'dateRanges': [{'startDate': '2021-01-01', 'endDate': '2021-02-28'}],
                            "pivots": [{"dimensions": [{"name": 'ga:yearMonth'}, {"name": "ga:channelGrouping"}],
                                        "metrics": [{"expression": "ga:users"},
                                                    {"expression": "ga:timeOnPage"}]
                                       }]
                          }]}

In [26]:
ga_report = run_report(body, ga_keys)
ga_report

(Other)               Organic Search  \
              ga:users ga:bounceRate  ga:users ga:bounceRate       ga:users   
ga:yearMonth                                                                  
202101        394726.0     46.066347  136896.0     54.950659        80483.0   
202102        287676.0     44.558331   80072.0     45.560776        60327.0   
202103        277777.0     45.235337  100887.0     52.206495        67385.0   
202104        262623.0     43.680061   59608.0     52.445146        67617.0   
202105        154821.0     37.779400   19723.0     45.666195        38581.0   
202106        166531.0     39.501808   17768.0     44.170585        35520.0   
202107        288840.0     43.388814   19049.0     41.501519        28265.0   

                             Direct               Paid Search                \
             ga:bounceRate ga:users ga:bounceRate    ga:users ga:bounceRate   
ga:yearMonth                                                                  
202101           37.069124  33574.0     40.985417     51906.0     52.069299   
202102           42.606405  26312.0     42.967781     80376.0     55.617553   
202103           38.992751  33075.0     43.064113     14603.0     50.043071   
202104           35.550721  43031.0     44.171972     27148.0     57.417219   
202105           33.565971  53769.0     34.019080      5552.0     43.421965   
202106           34.436494  68756.0     36.533087     11118.0     48.966711   
202107           35.228700  98625.0     41.400978     64592.0     48.412570   

               Social                Display               Referral  \
             ga:users ga:bounceRate ga:users ga:bounceRate ga:users   
ga:yearMonth                                                          
202101        59181.0     35.636420  26835.0     65.352831   5810.0   
202102        32052.0     26.364168   1513.0     58.082446   7013.0   
202103        32350.0     32.072254  16115.0     71.262027   7262.0   
202104        36820.0     33.244696   9570.0     68.795325   7102.0   
202105        17648.0     37.266922  14466.0     73.143406   3327.0   
202106        13508.0     40.774950  16976.0     65.394797   2644.0   
202107        53175.0     43.431733  18811.0     65.691801   3536.0   

                           Other Advertising                  Email  \
             ga:bounceRate          ga:users ga:bounceRate ga:users   
ga:yearMonth                                                          
202101           27.575422              41.0     35.714286      0.0   
202102           34.563410               9.0     44.444444      2.0   
202103           30.373070            6100.0     72.066090      0.0   
202104           27.810024           10477.0     73.270002   1250.0   
202105           23.681399              40.0     50.000000   1715.0   
202106           23.429912              87.0     55.905512    154.0   
202107           23.461153               6.0      7.142857   2781.0   

                            
             ga:bounceRate  
ga:yearMonth                
202101            0.000000  
202102          100.000000  
202103            0.000000  
202104           55.345912  
202105           49.150435  
202106           42.585551  
202107           51.014713